In [ ]:
# Autor: Elvis do A. Soares
# Github: @elvissoares
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import scienceplots
plt.style.use(['science','notebook'])

# Dinâmica Molecular no Ensemble NVE 

- $N$: número de partículas
- $V$: volume 
- $E$: energia


## Unidades reduzidas

Iremos usar unidades reduzidas

$$T^* = \frac{k_B T}{\epsilon}$$

$$\rho^* = \rho \sigma^2 $$

$$ t^* = \left( \frac{\epsilon}{m \sigma^2} \right)^{1/2} t$$

## Integrador Velocity-Verlet

Apropriado apenas para 2ª Lei de Newton. Utiliza um cálculo a mais de aceleração. 

$$x_{t+h} = x_t + v_t h + \frac{1}{2} a_t h^2$$

$$v_{t+h} = v_t + \frac{1}{2}(a_{t+h}+a_t) h $$

De modo que o algoritmo consiste em 

1. Calcula $x_{t+h}$ usando $v_t$ e $a_t$;
2. Calcula $a_{t+h}$ usando $x_{t+h}$;
3. Calcula $v_{t+h}$ para o próximo passo;
4. Volta ao passo 1;

Ref: https://pt.wikipedia.org/wiki/M%C3%A9todo_de_Verlet

Função que calcula o potencial de LJ entre pares

In [ ]:
def ulj(r,epsilon=1.0,sigma=1.0):
    return 4*epsilon*((sigma/r)**12-(sigma/r)**6)

Função que calcula a derivada do potencial de LJ entre pares

In [ ]:
def duljdr(r,epsilon=1.0,sigma=1.0):
    return -4*epsilon*(12*(sigma/r)**12-6*(sigma/r)**6)/r

In [ ]:
class LJsystem():
    # Função que cria a classe
    def __init__(self,N_atoms, m = 1.0, epsilon = 1.0, sigma = 1.0, rcut = 2.5):
        self.N_atoms = N_atoms

        # parametros dos atomos
        self.m = m*np.ones(self.N_atoms)
        self.sigma = sigma*np.ones(self.N_atoms)
        self.epsilon = epsilon*np.ones(self.N_atoms)

        # parametro de cutoff da interação de LJ
        self.rcut = rcut

        # Arrays de posições
        self.x = np.zeros(self.N_atoms)
        self.y = np.zeros(self.N_atoms)
        self.z = np.zeros(self.N_atoms)

        # Arrays de velocidade
        self.vx = np.zeros(self.N_atoms)        
        self.vy = np.zeros(self.N_atoms) 
        self.vz = np.zeros(self.N_atoms) 

        # Arrays de aceleração
        self.ax = np.zeros(self.N_atoms)        
        self.ay = np.zeros(self.N_atoms)
        self.az = np.zeros(self.N_atoms)

    def Set_Density(self,rho):
        self.rho = rho
        # calcular a caixa necessária 
        Vol = self.N_atoms/self.rho
        self.Lx = self.Ly = self.Lz = np.power(Vol,1/3.0)*self.sigma.max()

    def Initialize_Positions(self):
        # Número de pontos em x e y
        Nx = Ny = Nz = int(np.ceil(np.power(self.N_atoms,1/3.0)))

        # Espaçamento entre os átomos
        dx = self.Lx/Nx
        dy = self.Ly/Ny
        dz = self.Lz/Nz

        # Posições possíveis em x e y
        id = 0
        for i in range(Nx):
            for j in range(Ny):
                for k in range(Nz):
                    if (id < self.N_atoms):
                        self.x[id] = (i+0.5)*dx
                        self.y[id] = (j+0.5)*dy
                        self.z[id] = (k+0.5)*dz

                        id +=1

    def Initialize_Velocities(self,kT= 1.0):
        self.kT = kT

        # sorteia uma distribuição normal com devio padrão proporcional a temperatura
        self.vx[:] = np.random.normal(loc=0.0, scale=np.sqrt(self.kT/self.m),size=self.N_atoms)
        self.vy[:] = np.random.normal(loc=0.0, scale=np.sqrt(self.kT/self.m),size=self.N_atoms)
        self.vz[:] = np.random.normal(loc=0.0, scale=np.sqrt(self.kT/self.m),size=self.N_atoms)

        # retira qualquer movimento total
        self.vx[:] -= self.vx.mean()
        self.vy[:] -= self.vy.mean()
        self.vz[:] -= self.vz.mean()

    def Get_KineticEnergy(self):
        self.K = np.sum(0.5*(self.vx**2+self.vy**2+self.vz**2))
        return self.K

    def Calculate_Interactions(self):
        self.ax[:] = 0.0
        self.ay[:] = 0.0
        self.az[:] = 0.0
        self.U = 0.0
        for i in range(self.N_atoms):
            for j in range(i+1,self.N_atoms):
                # calcula distancia entre duas particulas
                rx = self.x[i] - self.x[j]
                ry = self.y[i] - self.y[j]
                rz = self.z[i] - self.z[j]
                # testa condição de contorno periodica
                rx = rx % self.Lx # em x
                ry = ry % self.Ly # em y
                rz = rz % self.Lz # em y
                # calcula modulo da distancia 
                r = np.sqrt(rx**2 + ry**2 + rz**2)
                # regra de combinação de Lorenz-Berthelot
                epsilonij = np.sqrt(self.epsilon[i]*self.epsilon[j])
                sigmaij = 0.5*(self.sigma[i]+self.sigma[j])
                # Vamos considerar somente interação dentro do raio de corte
                if r < self.rcut*sigmaij:
                    # calcula a derivada do potencial 
                    dudr = duljdr(r,epsilonij,sigmaij)
                    # calcula aceleração na particula i 
                    self.ax[i] += -dudr*rx/r
                    self.ay[i] += -dudr*ry/r
                    self.az[i] += -dudr*rz/r
                    # calcula aceleração na particula j usando 3ª Lei de Newton 
                    self.ax[j] += dudr*rx/r
                    self.ay[j] += dudr*ry/r
                    self.az[j] += dudr*rz/r
                    # calcula energia interna
                    self.U += ulj(r,epsilonij,sigmaij)

    def Get_PotentialEnergy(self):
        return self.U
    
    def Get_Energies(self):
        self.Get_KineticEnergy()
        kT = 2*self.K/(3*self.N_atoms)
        return self.K, self.U, kT
    
    def Set_TimeStep(self,h):
        self.h = h
    
    def Calculate_TimeStep(self):
        ...

    def RunSimulation(self,N_steps,Step_to_print=500):
        self.N_steps = N_steps
        self.Step_to_print = Step_to_print # steps to print output

        self.Calculate_Interactions()

        print('iter','K','U','T')
        print(0,self.Get_Energies())

        t = []
        K = []
        U = []
        E = []
        T = []

        for i in range(1,self.N_steps):

            self.Calculate_TimeStep()

            if i % self.Step_to_print == 0:
                Ktemp, Utemp, Ttemp = self.Get_Energies()
                t.append(i)
                K.append(Ktemp)
                U.append(Utemp)
                E.append(Ktemp+Utemp)
                T.append(Ttemp)
                print(i,self.Get_Energies())

        return t, K, U, E, T

In [ ]:
lj2d = LJsystem(N_atoms=125,m=1.0,epsilon=1.0,sigma=1.0)

In [ ]:
lj2d.Set_Density(0.2)

lj2d.rho, lj2d.Lx, lj2d.Ly, lj2d.Lz

In [ ]:
lj2d.Initialize_Positions()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(lj2d.x,lj2d.y,'o',ms=10.0,alpha=0.5)

plt.xlim(0,lj2d.Lx)
plt.ylim(0,lj2d.Ly)

plt.xlabel('x')
plt.ylabel('y')

In [ ]:
lj2d.Initialize_Velocities(kT=1.5) # equivalente a 300 K

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(lj2d.x,lj2d.y,'o',ms=10.0,alpha=0.5) # posição das partículas
plt.quiver(lj2d.x,lj2d.y,lj2d.vx,lj2d.vy,color='C0') # vetor de velocidade

plt.xlim(0,lj2d.Lx)
plt.ylim(0,lj2d.Ly)

plt.xlabel('x')
plt.ylabel('y')

In [ ]:
lj2d.Calculate_Interactions()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(lj2d.x,lj2d.z,'o',ms=10.0,alpha=0.5) # posição das partículas
plt.quiver(lj2d.x,lj2d.z,lj2d.ax,lj2d.az,color='r') # vetor de velocidade

plt.xlim(0,lj2d.Lx)
plt.ylim(0,lj2d.Ly)

plt.xlabel('x')
plt.ylabel('y')

In [ ]:
lj2d.Get_Energies()

In [ ]:
lj2d.Set_TimeStep(h=0.00003)

t, K, U, E, T = lj2d.RunSimulation(N_steps=5000)

Gráfico das posições iniciais e finais das partículas

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(lj2d.x,lj2d.y,'o',ms=10.0,alpha=0.5) # posição das partículas
plt.quiver(lj2d.x,lj2d.y,lj2d.vx,lj2d.vy,color='C0') # vetor de velocidade

plt.xlim(0,lj2d.Lx)
plt.ylim(0,lj2d.Ly)

plt.xlabel('x')
plt.ylabel('y')

In [ ]:
plt.plot(t,K,label='K')
plt.plot(t,U,label='U')
plt.plot(t,E,'k--',label='K+U')

plt.legend(loc='best')

plt.xlabel('t')
plt.ylabel(r'$E/\epsilon$')

In [ ]:
plt.plot(t,T,label='T')

plt.legend(loc='best')

plt.xlabel('t')
plt.ylabel(r'$k_B T/\epsilon$')

## Calculando Médias